In [19]:
import cobra.test
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import production_envelope, flux_variability_analysis, find_essential_reactions, find_essential_genes, pfba
import cobra.test
import os
import pandas as pd
import pickle
import numpy as np

In [82]:
WT_model = cobra.io.read_sbml_model("final_model/WT_model_final.xml")
HP_model = cobra.io.read_sbml_model("final_model/HP_model_final.xml")

<h3> pFBA simulation </h3>

In [5]:
model = cobra.io.read_sbml_model("final_model/WT_model_final.xml")
model.reactions.EX_glc__D_e.lower_bound = 0.0
model.reactions.EX_glyc_e.lower_bound = -0.8
model.reactions.DM_rap_c.lower_bound = 0.0005
model.objective = model.reactions.BIOMASS_SCO
solution = model.optimize()
print(solution.objective_value)
WT_pfba_solution = pfba(model)
print(WT_pfba_solution.fluxes['BIOMASS_SCO'])
print(WT_pfba_solution.objective_value)
#WT_pfba_solution.fluxes.to_csv('outputs/WT_pfba_fluxes.csv')

0.03367854092035982
0.03367854092035984
38.64520778598939


In [15]:
model = cobra.io.read_sbml_model("final_model/HP_model_final.xml")
model.reactions.EX_glc__D_e.lower_bound = 0.0
model.reactions.EX_glyc_e.lower_bound = -0.8
model.reactions.DM_rap_c.lower_bound = 0.0020
model.objective = model.reactions.BIOMASS_SCO
solution = model.optimize()
print(solution.objective_value)
HP_pfba_solution = pfba(model)
print(HP_pfba_solution.fluxes['BIOMASS_SCO'])
print(HP_pfba_solution.objective_value)
#HP_pfba_solution.fluxes.to_csv('outputs/HP_pfba_fluxes.csv')

0.022804391659545384
0.022804391659545294
37.88917595970602


In [16]:
count = 0
res = []
model = cobra.io.read_sbml_model("final_model/WT_model_final.xml")

for k,v in HP_pfba_solution.fluxes.items():
    HP_flux = v
    WT_flux = WT_pfba_solution.fluxes[k]
    
    if abs(HP_flux) < 1e-10:
        HP_flux = 0.0
    if abs(WT_flux) < 1e-10:
        WT_flux = 0.0
    
    '''
    HP_biomass = HP_pfba_solution.fluxes['BIOMASS_SCO']
    WT_biomass = WT_pfba_solution.fluxes['BIOMASS_SCO']
    
    norm_HP_flux = HP_flux/HP_biomass
    norm_WT_flux = WT_flux/WT_biomass
    
    if norm_WT_flux != 0.0:
        diff = abs(norm_HP_flux - norm_WT_flux)/norm_WT_flux
    elif norm_HP_flux != 0.0:
        diff = 1
    else:
        diff = 0
        
    if diff > 0.1:
        res.append([norm_WT_flux, norm_HP_flux, diff, k, model.reactions.get_by_id(k).reaction])
        count += 1
    '''    
    
    diff = HP_flux - WT_flux
    res.append([WT_flux, HP_flux, diff, k, model.reactions.get_by_id(k).reaction])
    
#print(count)
df = pd.DataFrame(res, columns =['WT_flux', 'HP_flux', 'diff', 'rxn_id', 'rxn'], dtype = float)
df
df.to_csv('outputs/pfba_flux_comparison_abs.csv')